In [ ]:
pip install pandas openpyxl


In [ ]:
from tkinter import filedialog
from tkinter import *
root = Tk()
root.withdraw()

file_path_data = filedialog.askopenfilename(title="Select file listing projects and students")
print(file_path_data)


In [ ]:
file_path_preferences = filedialog.askopenfilename(title="Select file listing student projects' preferences")
print(file_path_preferences)


In [ ]:
import pandas as pd

# Specify the path to the Excel file
#file_path_data = 'test.xlsx'


#-------------------------------------------------- 
# Read the 'Projects' sheet from the 'data' file into a pandas DataFrame
sheet_name = 'Projects'
try:
    data_projects = pd.read_excel(file_path_data, sheet_name=sheet_name)
    print("Data loaded successfully!")
except Exception as e:
    print(f"An error occurred: {e}")

# Display the contents of the DataFrame
#print(data)


# Get the content of the 'Project' column
col_name = 'Project'
if col_name in data_projects.columns:
    projects_column = data_projects[col_name]
    print(f"Contents of the {sheet_name}:")
    print(projects_column)
else:
    print(f"The column {col_name} does not exist in the sheet {sheet_name}")
    

#-------------------------------------------------- 
# Read the 'Students' sheet from the 'data' file into a pandas DataFrame
sheet_name = 'Students'
try:
    data_students = pd.read_excel(file_path_data, sheet_name=sheet_name)
    print("Data loaded successfully!")
except Exception as e:
    print(f"An error occurred: {e}")


# Get the contents of the 'Student' and 'Email' column
col_name = 'Student'
col_email = "Email"
if col_name in data_students.columns:
    students_column = data_students[col_name]
    print(f"Contents of the {sheet_name}")
    print(students_column)
else:
    print(f"The column {col_name} does not exist in the sheet {sheet_name}")   

if col_email in data_students.columns:
    emails_column = data_students[col_email]
    print(f"Contents of the {sheet_name}")
    print(emails_column)
else:
    print(f"The column {col_name} does not exist in the sheet {sheet_name}")      
    

#----------------------------------------------------------------
# Create new DataFrame to save preferences as table

# Define the column names
pref_columns = ['Student']
for project in projects_column:
    pref_columns.append(project)

print(pref_columns)

# Create an empty DataFrame with specified columns
pref_df = pd.DataFrame(columns=pref_columns)


#---------------------------------------------------------------- 
# Read the 'Sheet' sheet from the 'preferences' file into a pandas DataFrame
sheet_name = 'Sheet1'
try:
    pref_data = pd.read_excel(file_path_preferences, sheet_name=sheet_name)
    print("Student Preferences Data loaded successfully!")
except Exception as e:
    print(f"An error occurred: {e}")

#print(pref_data.columns)

pref_data.rename(columns={'Have you submitted a Thesis Self-Proposal?\n': 'Self-proposal',
                        'Please type your name\n' : 'TypedName',
                        'Do you wish to work in a pair': 'Pair',
                        'Please enter the name of your partner': 'Partner',
                        'Please select the project code of your 1st preferred research Topic (select NONE if you you have no preferred Topic)\n': 'Topic1',
                        'Please select the project code of your 2nd preferred research Topic (select NONE if you you have no more preferred Topic)\n': 'Topic2',
                        'Please select the project code of your 3rd preferred research Topic (select NONE if you you have no more preferred Topic)\n': 'Topic3',
                        'Have you had a meaningful discussion with the supervisors of those first three research Topics you have selected (by email, Teams or in-person)?': 'Discussion',
                        'Please select the project code of your 4th preferred research Topic (select NONE if you you have no more preferred Topic)\n': 'Topic4', 
                        'Please select the project code of your 5th preferred research Topic (select NONE if you you have no more preferred Topic).\n': 'Topic5',
                        'Please, select your 1st preferred Subject Area that you would be happy to work in (see Thesis Handbook for details).\n' : 'SubjectArea1',
                        'Please, select your 2nd preferred Subject Area that you would be happy to work in.\n' : 'SubjectArea2',
                        'Please, select your 3rd preferred Subject Area that you would be happy to work in.\n' : 'SubjectArea3',
                        'Select minimum 2 Types of project you would be happy to work on.' : 'ProjectTypes'},
                         inplace = True)

print(pref_data.columns)
               
#----------------------------------------------------------------
# Add data to preferences DataFrame

#for index, row in pref_data.iterrows():
for index, row in data_students.iterrows():

    student_name = row ['Student']
    student_email = row ['Email']
    
    print(student_email)    
    print(pref_data['Email'])
    
    pref_rows = pref_data.loc[pref_data['Email'] == student_email]
        
    if pref_rows.empty == False:
        
        print("pref_row: ", pref_rows)
        
        for index2, pref_row2 in pref_rows.iterrows():
            
            print("pref_row2: ", pref_row2)
        
            selfproposal = pref_row2['Self-proposal']

            if selfproposal == 'No':

                topics = [pref_row2['Topic1'], pref_row2['Topic2'], pref_row2['Topic3'], pref_row2['Topic4'], pref_row2['Topic5']]       
                areas = [pref_row2['SubjectArea1'], pref_row2['SubjectArea2'], pref_row2['SubjectArea3']]

                pref_student = pd.DataFrame({'Student' : [student_name],
                               'Email' : [student_email],
                               topics[0] : [1], topics[1] : [2], topics[2] : [3], topics[3] : [4], topics[4] : [5],
                               areas[0] : [6], areas[1] : [7], areas[2] : [8]})

    else:
        pref_student = pd.DataFrame({'Student': [student_name], 'Email': [student_email]})

    #print(pref_student)
    
    pref_df = pd.concat([pref_df, pref_student], ignore_index=True)

    
if 'NONE' in pref_df.columns:
    pref_df.drop(columns=['NONE'], inplace=True)

print(pref_df)

# Use ExcelWriter to write to the specified filename
with pd.ExcelWriter(file_path_data, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    pref_df.to_excel(writer, sheet_name='Student_preferences_python', index=False)

print(f"Sheet 'Student_preferences_python' has been successfully written to {file_path_data}")
